In [ ]:
# tripadvisor  destination webscraper
# author: Mohamed Wishan
# date: 31.12.2020
# © 2020 Mohamed Wishan, CC BY 4.0

# install required libraries
# intstall duplicated thrice to fix dependency errors
!pip install mysql-connector-python
!pip install requests-html

In [ ]:
# import required libraries
import time
import datetime
import mysql.connector
from requests_html import HTMLSession
from mysql.connector.constants import ClientFlag

In [ ]:
# set up mysql SSL connection
config = {
    'user': 'yourusername',
    'password': 'yourpassword',
    'host': 'youripaddress',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'ssl/server-ca.pem',
    'ssl_cert': 'ssl/client-cert.pem',
    'ssl_key': 'ssl/client-key.pem'
}
try:
    # add existing database to config dict
    config['database'] = 'webscraper'
    cnxn = mysql.connector.connect(**config)
    cursor = cnxn.cursor()
    print("Database connection successful")
except Exception as e:
    print("Could not connect to the database")
    print(e)

In [ ]:
try:
    # query a nonscraped destination to scrape
    cursor.execute("SELECT * FROM destination WHERE destination_scraped=0 LIMIT 1")
    out2 = cursor.fetchall()
    # print(out2[0][2])
    if bool(out2):
        url     = out2[0][2]
        session = HTMLSession()
        r       = session.get(url)
    else:
        print("No destination to scrape")
except Exception as e:
    print("Could not run query")
    print(e)


In [ ]:
for html in r.html:
    time.sleep(3)
    r = session.get(html.url)
    r_hotel_name = r.html.find('.listing_title')
    range_num = len(r_hotel_name)
    hotel_list = []

    for x in range(range_num):
        # HOTEL NAME
        if r_hotel_name[x].text[:9] == "Sponsored":
          hotel_name = r_hotel_name[x].text[10:]
        else: 
          hotel_name = r_hotel_name[x].text

        destination_id = out2[0][0]
        hotel_url = str(r_hotel_name[x].absolute_links)[2:-2]
        last_updated  = datetime.datetime.now()
        hotel_scraped = 0

        hotel_tuple = (destination_id, hotel_name, hotel_url, last_updated, hotel_scraped)
        hotel_list.append(hotel_tuple)
    # END OF SCRAPING PAGE

    # EXECUTE DATABASE CMD TO INSERT ALL HOTELS ON PAGE
    try:
        insert_hotel = ("INSERT INTO hotel (destination_id, hotel_name, hotel_page_url, last_updated, hotel_scraped) "
        "VALUES (%s, %s, %s, %s, %s )")
        cursor.executemany(insert_hotel, hotel_list)
        cnxn.commit()
        
        print("DB INSERT SUCCESSFUL")
        print(r.html)
    except Exception as e:
        print("COULD NOT INSERT HOTELS TO DATABASE")
        print("URL:")
        print(r.html)
        print(e)
# END OF ALL PAGES

try:
    #EXECUTE FINAL DATABASE QUERY TO UPDATE THE DESTINATION AS FULLY SCRAPED
    update_hotel = ("UPDATE destination SET destination_scraped=1 WHERE id=%s")
    cursor.execute(update_hotel, (out2[0][0],))
    cnxn.commit()
    cnxn.close()
    print("DESTINATION UPDATE SUCCESSFUL")
except Exception as e:
    print("SQL ERROR: COULD NOT UPDATE DESTINATION AS SCRAPED")
    print(e)

In [ ]:
try:
    #EXECUTE FINAL DATABASE QUERY TO UPDATE THE DESTINATION AS FULLY SCRAPED
    update_hotel = ("UPDATE destination SET destination_scraped=1 WHERE id=%s")
    cursor.execute(update_hotel, (out2[0][0],))
    cnxn.commit()
    cnxn.close()
    print("DESTINATION UPDATE SUCCESSFUL")
except Exception as e:
    print("SQL ERROR: COULD NOT UPDATE DESTINATION AS SCRAPED")
    print(e)